## DATASET CREATION

Unfortunately, the dataset I will work with does not come as one already done dataset - as it is mostly usual in Data Science projects. <br /> 
To get the sheer experience of how a normal Data Science job looks like I also wanted to dive into this issue. By experience I can tell that this takes a shit ton of time - like almost 80% - of preparing the data. <br /> 
Since the data usually shares the same countries and lists mostly the same years of the recording I used this as a primary key where I connect the datapoints with each other. <br /> 
However, the  countries are listed in rows along with the year of the recording - I want to have a final dataset that looks like follows: <br /> 

|Country | Afghanistan | Albania | ... | Zimbabwe | 
| ----- | ----------  | ------ | ----- | ------- | 
|Alcohol consumption [l] | 0.2 | 2.4 | ... | 0.01 |
|Human Develpment Index (HDI) | 0.1 | 0.15 | ... | 0.1 | 
|... | ... | ... | ... | ... |
|Healthcare Expenditure [$] | 13.322 | 15.211 | ... | 1.039 |

Thus, I have to transpose each of the countries and record each of the years as seperate entry in the dataset. <br /> 

All the data is [publicly available](https://ourworldindata.org), and this source is trusted by many notorious companies such as Vox, The Ney York times and even the top universities of this world like MIT, Oxford, Stanford. <br /> 
Hence, I assume that this data is rather based on actual recording from the respective country. <br /> 
Even the United Nation published their records in this page and I bet that these folks do some amazing work, which we can trust. <br />


But now let's not waste too much with the explaination part and go straight into how I merged the datafiles to one huge on. <br /> 

In [1]:
import os 
import sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
datapath = os.path.join(os.path.join(os.getcwd(), 'data'), 'Health')
datapath

'C:\\Users\\Lenny\\Documents\\Studium_Robotics (M.Sc.)\\03_Semester 3 - Oslo ERASMUS\\01_Applied Data Analysis and Machine Learning\\Project 3\\data\\Health'

In [3]:
dataFileNames = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f))]
type4Cols = []
type7Cols = []
manualLists = []
max_countries = 0
countries_list = []

# the datasets mostly have the same size of 4 columns and same setup so let's get those first

for file in dataFileNames: 
    try:
        df = pd.read_csv(os.path.join(datapath, str(file) ) , encoding='latin-1')
    except: 
        print(file)
        manualLists.append(file)
    if df.shape[1] == 4: # one type of files (4 columns) 
        type4Cols.append(file) 
        countries = list(set(df["Entity"]))
        if len(countries) > max_countries: 
            max_countries = len(countries)
            countries_list = countries
    elif df.shape[1] == 7: 
        type7Cols.append(file) 
    else: # manual shit  to do then 
        manualLists.append(file)
    
print( max_countries, countries_list)

C:\Users\Lenny\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


262 ['Sweden', 'High income', 'Kosovo', 'Sub-Saharan Africa (excluding high income)', 'United Kingdom', 'South Asia (IDA & IBRD)', 'Georgia', 'Greenland', 'Antigua and Barbuda', 'Hungary', 'Lebanon', 'Senegal', 'New Caledonia', 'Jamaica', 'Malta', 'French Polynesia', 'Bahrain', 'Burundi', 'Cyprus', 'Swaziland', 'Bahamas', 'North America', 'Puerto Rico', 'Finland', 'Kyrgyzstan', 'Middle income', 'Italy', 'Europe & Central Asia (IDA & IBRD)', 'Guyana', 'Laos', 'Mauritius', 'Panama', 'Middle East & North Africa (IDA & IBRD)', 'Singapore', 'Tanzania', 'Morocco', 'IDA only', 'Central African Republic', 'Malawi', 'Other small states', 'Djibouti', 'Mozambique', 'Macedonia', 'Australia', 'Barbados', 'Micronesia (country)', 'Samoa', 'Cuba', 'Malaysia', 'Bermuda', 'Aruba', 'Caribbean small states', 'Middle East & North Africa', 'Netherlands', 'Sub-Saharan Africa (IDA & IBRD)', 'Early-demographic dividend', 'Liechtenstein', 'British Virgin Islands', 'Indonesia', 'Cape Verde', 'Israel', 'Uganda', 

In [7]:
del countries_list[countries_list.index('IDA total')]
del countries_list[countries_list.index('Central Europe and the Baltics')]
del countries_list[countries_list.index('Latin America & Caribbean (excluding high income)')]
del countries_list[countries_list.index('East Asia & Pacific (excluding high income)')]
del countries_list[countries_list.index('South Asia (IDA & IBRD)')]
del countries_list[countries_list.index('Europe & Central Asia')]
del countries_list[countries_list.index('Middle East & North Africa (excluding high income)')]
del countries_list[countries_list.index('Europe & Central Asia (excluding high income)')]
del countries_list[countries_list.index('Heavily indebted poor countries (HIPC)')]
del countries_list[countries_list.index('Middle East & North Africa (IDA & IBRD)')]
del countries_list[countries_list.index('IDA blend')]
del countries_list[countries_list.index('Sub-Saharan Africa (IDA & IBRD)')]
del countries_list[countries_list.index('IDA & IBRD total')]
del countries_list[countries_list.index('South Asia')]
del countries_list[countries_list.index('Low & middle income')]
del countries_list[countries_list.index('East Asia & Pacific (IDA & IBRD)')]
del countries_list[countries_list.index('Early-demographic dividend')]

ValueError: 'IDA total' is not in list

In [8]:
## ADD MANUALLY THE LISTS WHICH ARE NOT IN THE MAXIMUM LIST 
countries_list.append('Syria')
countries_list.append('South Sudan')
countries_list.append('Niue')
max_countries = len(countries_list)
max_countries

248

In [ ]:
df.head(10)

In [17]:
# create dataframe where we want to paste everything inside
# 5.000.000 rows to not run into some problems while adding rows - delete later the other ones
final_df = pd.DataFrame(data = np.zeros( (3000000, max_countries) ), 
                        index = np.arange(3000000),
                        columns = [ country for country in countries_list])
# save indices as strings to get meaningful names
final_df.index = final_df.index.map(str)
final_df.shape

(3000000, 248)

In [ ]:
# testing to get values of the specific entries and Co
columns = list(df.columns)
b = df[df['Entity'] == 'France']
c = b[b['Year'] == 1999]#[columns[-1]]
d = b[b['Year'] == 1999][columns[-1]].sum()
print(d)
c
#df.groupby(["Entity", 'Year']).describe()

In [ ]:
%%time
# paste the values into the final_df from each single list
nextListIndex = 0

try: 
    for lists in type4Cols:
        
        ## Read the file and get the all countries along with their reported years
        print("{} list out of {}, Index: {}, Name: {}".format(type4Cols.index(lists), len(type4Cols), nextListIndex, lists))
        df = pd.read_csv(os.path.join(datapath, str(lists) ) , encoding='latin-1')
        
        # get the col names, unique countries and unique years
        columns = list(df.columns)
        countries = list(set(df["Entity"]))
        years = list(set(df['Year']))
        
        # get a list of all the index/row names
        indexNamesArr = final_df.index.values

        ## groupby countries and then years accordingly
        #df.groupby(["Entity", 'Year'])

        firstListRun = False # flag for renaming the indices

        # loop thru every country in the list
        for country in countries:
            
            ## check if country is in our masterlist
            if country not in countries_list: 
                # skip this item
                print("Country: {} not in list - but we skip it.".format(country))
                continue
            
            # take a dataframe for one country at a time
            country_df = df[df['Entity'] == country]

            # loop thru every year within that country - assuming the years are in the same order for every country
            for year in years:
                
                # rename the indices only if it is the very first run for the country
                if not firstListRun:
                    indexName = str(columns[-1]) + ' in ' + str(year)
                    indexNamesArr[nextListIndex] = indexName
                    nextListIndex += 1
                    if nextListIndex % 20 == 0: 
                        print("\t\t" + str(indexName))

                ## get the proper value and fill empty ones, if not available, fill it with the average
                # note: .sum() is only having one element anyway, just done to get the value as a float not an array
                value = country_df[country_df["Year"] == year][columns[-1]].sum() if not country_df[country_df["Year"] == year][columns[-1]].empty else country_df[columns[-1]].mean()
                # get the name of the row
                idxName = str(columns[-1]) + ' in ' + str(year)
                
                #print("Country: {} found in the dataset at spot: {}".format(country, countries_list.index(country)))
                
                # assign the value in the merged df with the value 
                final_df.iat[list(final_df.index.values).index(idxName), countries_list.index(country)] = value # .iat[row, col]

            ## get the proper index for the next list to begin with   
            firstListRun = True
            # print progress
            if countries.index(country) % 60 == 0:
                print("Working on country: {} out of {}".format(countries.index(country), len(countries)))
            
except Exception as e:
    print("Next List Index in line: {} of {}, Matrix Size: {}, list: {}, country: {}".format(i, nextListIndex, final_df.shape[0], lists, country))
    print(e)
    sys.exit()

0 list out of 60, Index: 0, Name: agricultural-area-per-capita.csv
Agriculutral are per capita [h/person] in 1980
Agriculutral are per capita [h/person] in 2000
Working on country: 0 out of 187
Country: Oceania not in list - but we skip it.


In [16]:
final_df[0:10]

Sweden  High income  Kosovo  \
Alcohol-attributable deaths [%] in 2012     3.3          0.0     0.0   
1                                           0.0          0.0     0.0   
2                                           0.0          0.0     0.0   
3                                           0.0          0.0     0.0   
4                                           0.0          0.0     0.0   
5                                           0.0          0.0     0.0   
6                                           0.0          0.0     0.0   
7                                           0.0          0.0     0.0   
8                                           0.0          0.0     0.0   
9                                           0.0          0.0     0.0   

                                         Sub-Saharan Africa (excluding high income)  \
Alcohol-attributable deaths [%] in 2012                                         0.0   
1                                                                               0.0   
2                                                                               0.0   
3                                                                               0.0   
4                                                                               0.0   
5                                                                               0.0   
6                                                                               0.0   
7                                                                               0.0   
8                                                                               0.0   
9                                                                               0.0   

                                         United Kingdom  Georgia  Greenland  \
Alcohol-attributable deaths [%] in 2012             3.4      4.6        0.0   
1                                                   0.0      0.0        0.0   
2                                                   0.0      0.0        0.0   
3                                                   0.0      0.0        0.0   
4                                                   0.0      0.0        0.0   
5                                                   0.0      0.0        0.0   
6                                                   0.0      0.0        0.0   
7                                                   0.0      0.0        0.0   
8                                                   0.0      0.0        0.0   
9                                                   0.0      0.0        0.0   

                                         Antigua and Barbuda  Hungary  \
Alcohol-attributable deaths [%] in 2012                  2.2      6.7   
1                                                        0.0      0.0   
2                                                        0.0      0.0   
3                                                        0.0      0.0   
4                                                        0.0      0.0   
5                                                        0.0      0.0   
6                                                        0.0      0.0   
7                                                        0.0      0.0   
8                                                        0.0      0.0   
9                                                        0.0      0.0   

                                         Lebanon  ...   Mexico  Kenya  \
Alcohol-attributable deaths [%] in 2012      1.5  ...      6.8    2.6   
1                                            0.0  ...      0.0    0.0   
2                                            0.0  ...      0.0    0.0   
3                                            0.0  ...      0.0    0.0   
4                                            0.0  ...      0.0    0.0   
5                                            0.0  ...      0.0    0.0   
6                                            0.0  ...      0.0    0.0   
7                               

In [ ]:
indexNames = final_df.index.values
idxName = 'Alcohol-attributable deaths [%] in 2012'
index = list(final_df.index.values).index(idxName)
#indexNames[50:65]
index

In [ ]:
final_df.loc[ 'b' : 'd'] #index it properly

In [ ]:
final_df.to_csv(os.path.join(datapath, "Merged UN Data.csv"))

BELOW HERE IS JUST THE PLAYGROUND TO TRY SOME STUFF OUT -> The dataset should be done in the top part

In [ ]:
df.groupby('Entity')['Wine Consumption'].sum().sort_values().tail(5)

In [ ]:
df[df['Entity'] == 'Germany']['Wine Consumption'].plot.hist(bins=20)